In [9]:
# read data from crest format
import pandas as pd
df = pd.read_excel('CNN/data/data_xlsx/simeval8.xlsx')
json_test = []
json_train = []

In [73]:
df.head()

,Unnamed: 0,original_id,span1,span2,signal,context,idx,label,direction,source,ann_file,split,global_id
0,0,1,['configuration'],['elements'],[],The system as described above has its greatest...,span1 73:86\nspan2 98:106\nsignal,0,1,2,NaN,0,1
1,1,2,['child'],['cradle'],[],The child was carefully wrapped and bound into...,span1 4:9\nspan2 51:57\nsignal,0,-1,2,NaN,0,2
2,2,3,['author'],['disassembler'],[],The author of a keygen uses a disassembler to ...,span1 4:10\nspan2 30:42\nsignal,0,1,2,NaN,0,3
3,3,4,['ridge'],['surge'],[],A misty ridge uprises from the surge.,span1 8:13\nspan2 31:36\nsignal,0,-1,2,NaN,0,4
4,4,5,['student'],['association'],[],The student association is the voice of the un...,span1 4:11\nspan2 12:23\nsignal,0,0,2,NaN,0,5


In [74]:
# convert crest into cnn format
def find_index_in_sentence(sentence, searched_word):
    for i, word in enumerate(sentence):
        if word == searched_word:
            return i
    return -1

def extract_start_end_position(sentence, span):
    # sentence = remove_special_characters(sentence) 
    # span = remove_special_characters(span) 
    span = span.split()
    if len(span) == 0:
        return -1, -1
    else:
        start = find_index_in_sentence(sentence, span[0])
    if len(span) > 1:
        end = find_index_in_sentence(sentence, span[-1]) 
    else:
        end = start
    return start, end 

def remove_special_characters(sentence): 
    specialChars = "!#$%^&*()-,." 
    for specialChar in specialChars:
        sentence = sentence.replace(specialChar, ' ')
    sentence = ' '.join(sentence.split())
    return sentence
    

for index, row in df.iterrows():
    line = {}
    line['id'] = str(row['global_id']) 
    if row['label'] == 1:
        if row['direction'] == 0:
            line['relation'] = "Cause-Effect(e1,e2)"
        else:
            line['relation'] = "Cause-Effect(e2,e1)"
    else:
        line['relation'] = "Other"
    row['context'] = remove_special_characters(row['context'])
    sentence = row['context'].split(" ")
    sentence.append(".") 
    span1, span2 = row['span1'][2:-2], row['span2'][2:-2]
    line['head'] = remove_special_characters(span1)
    line['tail'] = remove_special_characters(span2)
    line['subj_start'], line['subj_end'] = extract_start_end_position(sentence, line['head']) 
    line['obj_start'], line['obj_end'] = extract_start_end_position(sentence, line['tail'])  
    line['sentence'] = sentence
    if row['split'] == 0:
        json_train.append(line)
    elif row['split'] == 2:
        json_test.append(line)

In [75]:
# save train.json; put it into CNN/data/
import json
with open("train2010.json", "w") as json_file:
    for line in json_train:
        json_file.write(json.dumps(line))
        json_file.write("\n")

In [76]:
# save test.json; put it into CNN/data/
with open("test2010.json", "w") as json_file:
    for line in json_test:
        json_file.write(json.dumps(line))
        json_file.write("\n")